In [1]:
from datetime import datetime
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from markdownify import markdownify 
import folium.plugins as plugins
import folium
import io




# endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"

endpoint_url = "https://query.wikidata.org/sparql"

query = """#defaultView:Map

SELECT DISTINCT ?item ?itemLabel ?geo ?latitude ?longitude ?locationLabel ?website ?street (YEAR(?inception) AS ?year) WHERE {
 VALUES ?type { wd:Q1007870 wd:Q33506 wd:Q166118 wd:Q56190453 }
  #art gallery, museum, archive, memorial
  
  ?item wdt:P31/wdt:P279* ?type; # Must be an instance of a relevant cultural type
        (wdt:P276|wdt:P131)* wd:Q1715.  # Located in Hannover or a part of Hannover (or a part of a part of Hannover...) 
  
  OPTIONAL { ?item wdt:P625 ?geo }
  OPTIONAL { ?item wdt:P131 ?location } 
  OPTIONAL { ?item wdt:P6375 ?street }
  OPTIONAL { ?item wdt:P856 ?website }
  OPTIONAL { ?item wdt:P571 ?inception }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "de,[AUTO_LANGUAGE]". }

  BIND(geof:latitude(?geo) AS ?latitude)
  BIND(geof:longitude(?geo) AS ?longitude)
}

ORDER BY DESC(?geo)
"""



def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

locations = []

def get_items():

    q = query

    results = run_query(endpoint_url, q)

    for item in results["results"]["bindings"]:    

      title=item['itemLabel']['value'] if 'itemLabel' in item else ''
      year=item['year']['value'] if 'year' in item else ''
      latitude=item['latitude']['value'] if 'latitude' in item else ''
      longitude=item['longitude']['value'] if 'longitude' in item else ''
      place=item['locationLabel']['value'] if 'locationLabel' in item else ''
      item_url=item['item']['value'] if 'item' in item else ''
      website=item['website']['value'] if 'website' in item else ''
      street=item['street']['value'] if 'street' in item else ''

      locations.append((title, latitude, longitude, website, item_url, place, street, year))






In [2]:
get_items()

m_hannover = folium.Map(location=[52.38, 9.74], zoom_start=12)

folium.plugins.Fullscreen(
    position="topright",
    title="Vollbild",
    title_cancel="Schließen",
    force_separate_button=True,
).add_to(m_hannover)

for name, lat, lon, website, item, place, street, year in locations:
    if not lat or not lon:  # Überspringen, wenn lat oder lon leer sind
        continue
    
    folium.Marker(
        location=[lat, lon],
        popup=f"""<h3><a href='{item}' target='_blank'>{name}</a></h3>
        {place}""",
        tooltip=name  # Hover label
    ).add_to(m_hannover)

print("""
[Wikidata Link](https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Fgeo%20%3Flatitude%20%3Flongitude%20%3FlocationLabel%20%3Fwebsite%20%3Fstreet%20%28YEAR%28%3Finception%29%20AS%20%3Fyear%29%20WHERE%20%7B%0A%20VALUES%20%3Ftype%20%7B%20wd%3AQ1007870%20wd%3AQ33506%20wd%3AQ166118%20wd%3AQ56190453%20%7D%0A%20%20%23art%20gallery%2C%20museum%2C%20archive%2C%20memorial%0A%20%20%0A%20%20%3Fitem%20wdt%3AP31%2Fwdt%3AP279%2a%20%3Ftype%3B%20%23%20Must%20be%20an%20instance%20of%20a%20relevant%20cultural%20type%0A%20%20%20%20%20%20%20%20%28wdt%3AP276%7Cwdt%3AP131%29%2a%20wd%3AQ1715.%20%20%23%20Located%20in%20Hannover%20or%20a%20part%20of%20Hannover%20%28or%20a%20part%20of%20a%20part%20of%20Hannover...%29%20%0A%20%20%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP625%20%3Fgeo%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP131%20%3Flocation%20%7D%20%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP6375%20%3Fstreet%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP856%20%3Fwebsite%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP571%20%3Finception%20%7D%0A%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22de%2C%5BAUTO_LANGUAGE%5D%22.%20%7D%0A%0A%20%20BIND%28geof%3Alatitude%28%3Fgeo%29%20AS%20%3Flatitude%29%0A%20%20BIND%28geof%3Alongitude%28%3Fgeo%29%20AS%20%3Flongitude%29%0A%7D%0A%0AORDER%20BY%20DESC%28%3Fgeo%29)
""")



[Wikidata Link](https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Fgeo%20%3Flatitude%20%3Flongitude%20%3FlocationLabel%20%3Fwebsite%20%3Fstreet%20%28YEAR%28%3Finception%29%20AS%20%3Fyear%29%20WHERE%20%7B%0A%20VALUES%20%3Ftype%20%7B%20wd%3AQ1007870%20wd%3AQ33506%20wd%3AQ166118%20wd%3AQ56190453%20%7D%0A%20%20%23art%20gallery%2C%20museum%2C%20archive%2C%20memorial%0A%20%20%0A%20%20%3Fitem%20wdt%3AP31%2Fwdt%3AP279%2a%20%3Ftype%3B%20%23%20Must%20be%20an%20instance%20of%20a%20relevant%20cultural%20type%0A%20%20%20%20%20%20%20%20%28wdt%3AP276%7Cwdt%3AP131%29%2a%20wd%3AQ1715.%20%20%23%20Located%20in%20Hannover%20or%20a%20part%20of%20Hannover%20%28or%20a%20part%20of%20a%20part%20of%20Hannover...%29%20%0A%20%20%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP625%20%3Fgeo%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP131%20%3Flocation%20%7D%20%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP6375%20%3Fstreet%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fitem

In [3]:

m_hannover


In [4]:
print("""
```{=html}
      </br>
      <table class='table table-striped' id='mapTable'>
      <thead>
      <tr>
      <th style="width:35%"><b onclick="sortTable(0)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Name</th>
      <th><b onclick="sortTable(1)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Koordinaten</th>
      <th style="width:10%"><b onclick="sortTable(2)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Straße</th>
      <th><b onclick="sortTable(3)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Ort</th>
      <th><b onclick="sortTable(4)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Web</th>
      <th><b onclick="sortTable(5)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Gründung</th>
      </tr>
      </thead>
      <tbody>
```
      """)
for name, lat, lon, website, item, place, street, year in locations:
  print(f"""
```{{=html}}
  <tr>
  <td><span style='display:none'>{name}</span><a href='{item}'>{name}</a></td>
```
  """)
  if lat == '' and lon == '':
    print("""
```{=html}
  <td></td>
```
""")
  else:
    print(f"""
```{{=html}}
  <td>{lat},<br> {lon}</td>
```
""")
  print(f"""
```{{=html}}
<td>{street}</td>
```
""")
  print(f"""
```{{=html}}
  <td>{place}</td>
```
""")
  if website == '':
    print("""
```{=html}
<td></td>
```
""")
  else:
    print(f"""
```{{=html}}
<td><a style="text-decoration:none;color:#df382c;font-size:big" class="bi bi-link-45deg" href="{website}"></a></td>
```
""")
  print(f"""
```{{=html}}
<td>{year}</td>
```
""")
  print("</tr>")
print("""
```{=html}
</tbody></table>
```
""")


print("""
<script>
function sortTable(n) {
  var table, rows, switching, i, x, y, shouldSwitch, dir, switchcount = 0;
  table = document.getElementById("mapTable");
  switching = true;
  // Set the sorting direction to ascending:
  dir = "asc";
  /* Make a loop that will continue until
  no switching has been done: */
  while (switching) {
    // Start by saying: no switching is done:
    switching = false;
    rows = table.rows;
    /* Loop through all table rows (except the
    first, which contains table headers): */
    for (i = 1; i < (rows.length - 1); i++) {
      // Start by saying there should be no switching:
      shouldSwitch = false;
      /* Get the two elements you want to compare,
      one from current row and one from the next: */
      x = rows[i].getElementsByTagName("TD")[n];
      y = rows[i + 1].getElementsByTagName("TD")[n];
      /* Check if the two rows should switch place,
      based on the direction, asc or desc: */
      if (dir == "asc") {
        if (x.innerHTML.toLowerCase() > y.innerHTML.toLowerCase()) {
          // If so, mark as a switch and break the loop:
          shouldSwitch = true;
          break;
        }
      } else if (dir == "desc") {
        if (x.innerHTML.toLowerCase() < y.innerHTML.toLowerCase()) {
          // If so, mark as a switch and break the loop:
          shouldSwitch = true;
          break;
        }
      }
    }
    if (shouldSwitch) {
      /* If a switch has been marked, make the switch
      and mark that a switch has been done: */
      rows[i].parentNode.insertBefore(rows[i + 1], rows[i]);
      switching = true;
      // Each time a switch is done, increase this count by 1:
      switchcount ++;
    } else {
      /* If no switching has been done AND the direction is "asc",
      set the direction to "desc" and run the while loop again. */
      if (switchcount == 0 && dir == "asc") {
        dir = "desc";
        switching = true;
      }
    }
  }
}
</script>
""") 


```{=html}
      </br>
      <table class='table table-striped' id='mapTable'>
      <thead>
      <tr>
      <th style="width:35%"><b onclick="sortTable(0)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Name</th>
      <th><b onclick="sortTable(1)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Koordinaten</th>
      <th style="width:10%"><b onclick="sortTable(2)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Straße</th>
      <th><b onclick="sortTable(3)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Ort</th>
      <th><b onclick="sortTable(4)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Web</th>
      <th><b onclick="sortTable(5)" style="color: #df382c; cursor:pointer">⭿</b>&nbsp;Gründung</th>
      </tr>
      </thead>
      <tbody>
```
      

```{=html}
  <tr>
  <td><span style='display:none'>Münzkabinett der TUI AG</span><a href='http://www.wikidata.org/entity/Q76636159'>Münzkabinett der TUI AG</a></td>
```
  

```{=html}
  <td>52.384001,<br> 9.811558</td>
